# Prep

In [ ]:
import os
import shutil
import random
import cv2
import numpy as np

In [ ]:
weightFile = "/content/drive/MyDrive/yolov3/FinalPartial/FinalPartial.weights"
configFile = "/content/drive/MyDrive/yolov3/FinalPartial/yolov3_training.cfg"
namesFile = "/content/drive/MyDrive/yolov3/FinalPartial/obj.names"
testImageLocation = '/content/drive/MyDrive/yolov3/testImage.jpg'
testImageLocation2 = '/content/drive/MyDrive/yolov3/testImage2.jpg'

In [ ]:
from google.colab.patches import cv2_imshow #cv2.imshow is broken in google colab therefore use this as replacement

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Define Recognition

In [ ]:
def recognizeFace(fileLocation):
  net = cv2.dnn.readNet(weightFile, configFile) #pass in trained model weights and configuration file
  classes = [] #store classes here

  #load object class names
  with open(namesFile, "r") as f:
    classes = f.read().splitlines()

  print(classes) #should be 4

  #load test image
  img = cv2.imread(fileLocation)
  height, width, _ = img.shape

  #prep image blob
  blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False) #1/255 is scaling

  net.setInput(blob) #input blob into network
  output_layers_names = net.getUnconnectedOutLayersNames()
  layerOutputs = net.forward(output_layers_names)

  boxes = []
  confidences = []
  class_ids = []

  for output in layerOutputs:
    for detection in output:
      scores = detection[5:] #starting from the sixth element to the end
      class_id = np.argmax(scores) #extract highest score location
      confidence = scores[class_id]
      
      if confidence > 0.5:
        center_x = int(detection[0]*width)
        center_y = int(detection[1]*height)
        w = int(detection[2]*width)
        h = int(detection[3]*height)

        #upper left corner
        x = int(center_x - w/2)
        y = int(center_y - h/2)

        boxes.append([x, y, w, h])
        confidences.append(float(confidence))
        class_ids.append(class_id)

  indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  font = cv2.FONT_HERSHEY_PLAIN
  colors = np.random.uniform(0, 255, size=(len(boxes), 3))

  if(len(indexes) > 0):
    for i in indexes.flatten():
      x, y, w, h = boxes[i]
      label = str(classes[class_ids[i]])
      confidence = str(round(confidences[i], 2))
      color = colors[i]
      cv2.rectangle(img, (x, y), (x+w, y+h), color, 2) #thickness is 2
      cv2.putText(img, label + " " + confidence, (x, y + 20), font, 2, (255, 0, 0), 2)

  else:
    print("Indexes are empty!")

  #display image
  cv2_imshow(img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

#Execution

In [ ]:
recognizeFace(testImageLocation)
#recognizeFace(testImageLocation2)